# NEUFEP-ME study: Data setup workflow

This notebook contains the workflow required to convert all raw data into the formats, types and structures necessary for the main data processing pipeline. Thus, getting data ready to be processed.

## Study overview

The main purpose of the study is to investigate methods to improve the quality of real-time functional magnetic resonance imaging (fMRI) data. These improvements are for future applications in real-time fMRI neurofeedback, a method where participants are presented with visual feedback of their brain activity while they are inside the MRI scanner, and then asked to regulate the level of feedback. We have developed real-time multi-echo EPI acquisition sequences and processing methods, and this study aims to collect data from volunteers in order to validate these new methods. No neurofeedback is provided.

```
The scan session will consist of a number of scan sequences, some of which will require you to look at pictures or perform a task, and others just to lie still in the scanner. The total time in the scanner will be around 1 hour, with a break in the middle. Volunteers have to be right-handed, healthy and should have no history of or be under current treatment for psychiatric / neurological conditions. If this inclusion criteria do not fit you, you will unfortunately not be able to participate in the study. Other reasons for not being able to participate include being pregnant or having metal implants.
```

## Data

Per subject, all of the following data were collected during one scan session (all functional scans are multi-echo EPI):

| Nr | Name  | Scan Type | Description | Format |
| :--- | :--- | :--- | :--- | :--- |
| 1 | T1w | Anatomical | Standard T1-weighted sequence | NIfTI |
| 2 | run1_BOLD_rest | Functional | Resting state | PAR/REC, XML/REC, DICOM |
| 3 | run1_BOLD_task1 | Functional | Motor - finger tapping | PAR/REC, XML/REC, DICOM |
| 4 | run1_BOLD_task2 | Functional | Emotion - shape/face matching | PAR/REC, XML/REC, DICOM |
| 5 | run2_BOLD_rest | Functional | Resting state | PAR/REC, XML/REC, DICOM |
| 6 | run2_BOLD_task1 | Functional | Motor mental - imagined finger tapping | PAR/REC, XML/REC, DICOM |
| 7 | run2_BOLD_task2 | Functional | Emotion mental - recalling emotional memories | PAR/REC, XML/REC, DICOM |
| 8 | Stimulus timing | Peripheral measure | Stimulus and response timing for all tasks, i.e. x4 | Eprime .dat and .txt |
| 9 | Physiology | Peripheral measure | Cardiac + respiratory traces for all runs, i.e. x6 | Philips "scanphyslog" |

## Data setup goals

For each dataset (i.e. for each subject) we have to:

1. Move all files into a machine readable directory structure
2. Rename all image files in this directory structure such that BIDS tags are findable
3. Convert data to BIDS:
  1. Run `bidsify` to convert the image data to BIDS (This includes conversion of PAR/REC to NIfTI using `dcm2niix`; this also includes anonymization, which doesn't work for some reason)
  2. Deface the T1w image using `pydeface`
  3. Run eprime conversion script to convert stimilus and response timings to BIDS (need to figure out this format)
  4. Run `scanphyslog2bids` (or Matlab script if needed) to convert physiology data to BIDS
4. Run the BIDS validator
6. Create summary tables and plots using `pybids`

## Required packages / software

- bidsify: https://github.com/NILAB-UvA/bidsify/tree/master/bidsify
- pydeface: https://github.com/poldracklab/pydeface
- fsl (required for pydeface): https://fsl.fmrib.ox.ac.uk/fsl/fslwiki
- bids_validator: https://github.com/bids-standard/bids-validator; https://bids-standard.github.io/bids-validator/
- convert-eprime: https://github.com/tsalo/convert-eprime
- scanphyslog2bids: https://github.com/lukassnoek/scanphyslog2bids

In [1]:
%matplotlib inline
import os
import shutil
import glob
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import bidsify
import pydeface
import pandas as pd
from bids_validator import BIDSValidator
from convert_eprime.convert import text_to_csv

## Step 0: initialize variables, directories, filenames, etc


In [2]:
# Dependent on structure and filenames in current data on external drive
data_dir = '/Volumes/Stephan_WD/NEUFEPME_data/'
new_data_dir = '/Volumes/Stephan_WD/NEUFEPME_data/NEUFEPME_data_new/'
org_data_dir = '/Volumes/Stephan_WD/NEUFEPME_data_organised/'
new_org_data_dir = org_data_dir + 'new/'
bids_data_dir = '/Volumes/Stephan_WD/NEUFEPME_data_BIDS/'
eprime_data_dir = '/Volumes/Stephan_WD/NEUFEPME_data_eprime/'

## Step 1+2: Move and rename all files into a machine readable and BIDS-ready directory structure


#### ONCE ONLY: copy all raw data, with exceptions, to new directory (leave raw data untouched!)


In [4]:
# Ignore files:
# - with 'FSL', this is a proxy for the FSL '.nii' file
# - with spaces (this is a proxy for the xml/rec files),
# - the PAR/RECs of T1w file
src = new_data_dir
dst = new_org_data_dir
ignore_pattern = shutil.ignore_patterns('*FSL*',
                                        '* *',
                                        '*T1W*.PAR',
                                        '*T1W*.REC')
shutil.copytree(src, dst, ignore=ignore_pattern)

'/Volumes/Stephan_WD/NEUFEPME_data_organised/new/'

#### ONCE ONLY: Rename files such that bidsify can read separate tags and convert correctly to BIDS format
See filenames in: "/Users/jheunis/Documents/PYTHON/rtme-fMRI/bidsify_test_2709/sub-01")


In [7]:
# This means removing the 'SENSE' text from the T1w file such that files with SENSE are all recognised as BOLD
os.chdir(new_org_data_dir)
t1w = glob.glob('*/*T1W*',recursive=True)
for fn in t1w:
    os.rename(fn, fn.replace('_SENSE', ''))

#### ONCE ONLY: Rename scanphyslog files based on time of acquisition and relation to functional scans

In [21]:
# # Typically 10 files in each subject's 'scanphyslog' directory. If not, throw error.
# scan_names = ['_emotion2', '_motor2', '_rest2', '_emotion1', '_motor1', '_rest1']
# all_subs = next(os.walk(new_org_data_dir))[1]

# print(all_subs)
# for sub in all_subs:
#     print(sub)
#     os.chdir(new_org_data_dir + sub + '/scanphyslog')
#     files = glob.glob('*')
#     files.sort(key=os.path.getmtime, reverse=True)
#     if len(files) != 10:
#         print('ERROR: there are {} files. Not renaming files'.format(len(files)))
#     else:
#         for i, file in enumerate(files):
#             if i < 6:
#                 new_file = os.path.splitext(file)[0] + scan_names[i] + os.path.splitext(file)[1]
#                 print(new_file)
#                 os.rename(file, new_file)
#             else:
#                 print(file)

## Step 3.1: Convert image data to BIDS

Using `bidsify`

See `me_workflow.ipynb`


In [20]:
# !cd /Volumes/Stephan_WD/NEUFEPME_data_organised/new && bidsify -v

## Step 3.2: Deface T1w images

Using `pydeface`


In [19]:
# os.chdir(bids_data_dir)
# all_subs = next(os.walk(bids_data_dir))[1]
# print(all_subs)

# for i, sub in enumerate(all_subs):
#     anat_dir = os.path.join(bids_data_dir, sub, 'anat')
#     T1w_fn = os.path.join(anat_dir, sub+'_T1w.nii')
#     T1w_defaced_fn = os.path.join(anat_dir, sub+'_T1w_defaced.nii')
#     if not os.path.exists(T1w_defaced_fn):
#         print('{} - {}'.format(i, T1w_fn))
#         #!cd $anat_dir && pydeface $T1w_fn

## Step 3.3: Convert Eprime data to BIDS

Use `neufepme_read_eprime.ipynb`, which uses `convert-eprime`

see: https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/05-task-events.html

Perhaps write this into separate functions?


In [3]:
import utilities as util

paradigms = ['motor-', 'motor-imagine-', 'emotion-', 'emotion-imagine-']
paradigm_nrs = ['-001', '-002', '-001', '-002']
event_names = ['_task-motor_run-1_', '_task-motor_run-2_', '_task-emotion_run-1_', '_task-emotion_run-2_']
keys = ['df0', 'df1', 'df2', 'df3']
dct_df = dict.fromkeys(keys, None)

# Loop through all subjects in BIDS folder
os.chdir(bids_data_dir)
all_subs = next(os.walk(bids_data_dir))[1]

sub = 'sub-012'
# for sub in all_subs:
# Find text files for each subject's four runs (see `paradigms` for run types)
# Create empty csv files (into which text file content is copied) and tsv files (for events in BIDS format)
sub_data_dir = os.path.join(eprime_data_dir, sub)
txt_files = [None] * 4
csv_files = [None] * 4
event_files = [None] * 4
sub_nr = sub.split('-')[1]
for i, par in enumerate(paradigms):
    txt_files[i] = os.path.join(sub_data_dir, 'neufepme-' + par + sub_nr + paradigm_nrs[i] + '.txt')
    name, ext = os.path.splitext(txt_files[i])
    csv_files[i] = name + '.csv'
    event_files[i] = os.path.join(bids_data_dir, sub, 'func', sub+event_names[i]+'events.tsv')
# Read text files, convert to csv files, and cv files into dataframes 
all_dfs = {}
for i, fn in enumerate(txt_files):
    text_to_csv(fn, csv_files[i])
    df = pd.read_csv(csv_files[i])
    all_dfs[paradigms[i]] = df
        
dct_df = util.create_event_files(all_dfs, paradigms, dct_df)
pd.set_option('display.max_rows', None)

## FIX ALL THE DATAFRAMES TO MAKE THEM BIDS COMPATIBLE

# Drop the 'Rest' rows from df0, df1, df3
for i in [0,1,3]:
    df = dct_df[keys[i]]
    idx = df[df['trial_type'] == 'Rest'].index
    df.drop(idx , inplace=True)

# Update df2:
# 'button_pressed' = NaN ==> n/a; same rows 'response_time' ==> n/a
df = dct_df['df2']
nan_rows = df[df['button_pressed'].isnull()]
df.loc[nan_rows.index, 'response_time'] = 'n/a'
df.loc[nan_rows.index, 'button_pressed'] = 'n/a'
# r = red = RIGHT ==> R; g = green = LEFT ==> L
r_rows = df[df['button_pressed'] == 'r']
df.loc[r_rows.index, 'button_pressed'] = 'R'
g_rows = df[df['button_pressed'] == 'g']
df.loc[g_rows.index, 'button_pressed'] = 'L'
# Fix onset times
df.loc[0, 'onset'] = 0.000
for i in df.index:
    if i > 0:
        df.loc[i, 'onset'] = np.round(df.loc[i-1, 'onset'] + df.loc[i-1, 'duration'], 3)

print(dct_df)
for i, key in enumerate(keys):
    events = dct_df[key]
    events.to_csv(event_files[i], sep='\t', index=False)

Output file successfully created- /Volumes/Stephan_WD/NEUFEPME_data_eprime/sub-012/neufepme-motor-012-001.csv
Output file successfully created- /Volumes/Stephan_WD/NEUFEPME_data_eprime/sub-012/neufepme-motor-imagine-012-002.csv
Output file successfully created- /Volumes/Stephan_WD/NEUFEPME_data_eprime/sub-012/neufepme-emotion-012-001.csv
Output file successfully created- /Volumes/Stephan_WD/NEUFEPME_data_eprime/sub-012/neufepme-emotion-imagine-012-002.csv
{'df0':       onset duration     trial_type
1    20.013       20  FingerTapping
3    60.038       20  FingerTapping
5   100.063       20  FingerTapping
7   140.089       20  FingerTapping
9   180.114       20  FingerTapping
11  220.140       20  FingerTapping
13  260.165       20  FingerTapping
15  300.190       20  FingerTapping
17  340.216       20  FingerTapping
19  380.241       20  FingerTapping, 'df1':       onset duration           trial_type
1    20.012       20  MentalFingerTapping
3    60.038       20  MentalFingerTapping
5 

## Step 3.4: Convert physiology data to BIDS

Using `scanphyslog2bids`

See: https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/06-physiological-and-other-continuous-recordings.html


In [5]:
from scanphyslog2bids.core import PhilipsPhysioLog
import nibabel as nib
import numpy as np

main_dir = '/Users/jheunis/Desktop/sample-data/sub-neufepmetest/sub-pilot/raw-data'
log_file = main_dir + '/SCANPHYSLOG20191011115018_motor1.log'
out_dir = main_dir + '/my_bids_data'  # where the BIDSified data should be saved
deriv_dir = main_dir + '/my_bids_data/physio'  # where some QC plots should be saved

# fmri_file is used to extract metadata, such as TR and number of volumes
fmri_file = main_dir + '/sub-pilot_task-motor_run-1_echo-2.nii' 
fmri_img = nib.load(fmri_file)
n_dyns = fmri_img.shape[-1]
tr = np.round(fmri_img.header['pixdim'][4], 3)

# Create PhilipsPhysioLog object with info
phlog = PhilipsPhysioLog(f=log_file, tr=tr, n_dyns=n_dyns, sf=500, manually_stopped=False)

# Load in data, do some preprocessing
phlog.load()

# Try to align physio data with scan data, using a particular method
# (either "vol_markers", "gradient_log", or "interpolation")
# phlog.align(trigger_method='vol_markers')  # load and find vol triggers
phlog.align(trigger_method='interpolation')  # load and find vol triggers

# Write out BIDS files
phlog.to_bids(out_dir)  # writes out .tsv.gz and .json files

# Optional: plot some QC graphs for alignment and actual traces
phlog.plot_alignment(out_dir=deriv_dir)  # plots alignment with gradient
phlog.plot_traces(out_dir=deriv_dir)  # plots cardiac/resp traces

2020-01-09 14:04:33,301 [MainThread] [INFO   ]  Found end marker ('0020') at an offset of 32 samples relative to the end of the file.
2020-01-09 14:04:37,376 [MainThread] [INFO   ]  Trimmed off 59 samples from end of file based on the (absence) of a gradient.


CouldNotFindThresholdError: Could not find threshold!

## Step 3.5: Add slice-timing information

In [ ]:
# First run scripts to ensure that BIDS json data is correct
TR = 2
N_slices = 34
t_d = TR/N_slices
t_start = 0
t_stop = TR
slice_timing = np.arange(start=t_start, stop=t_stop, step=t_d).round(4).tolist()

## Step 4: Run BIDS validator

see: https://bids-standard.github.io/bids-validator/

In [6]:
# Check if possible to run validator on a full directory tree

# fn = '/Users/jheunis/Documents/PYTHON/rtme-fMRI/bidsify_test_2709/sub-01/anat/sub-01_T1w.nii'
# BIDSValidator().is_bids(fn)



## Step 6: Create summary tables and plots using `pybids`

See:
- https://bids-standard.github.io/pybids/
- https://github.com/bids-standard/pybids/blob/master/examples/pybids_tutorial.ipynb
- https://mybinder.org/v2/gh/bids-standard/pybids/master

this is also useful for de/constructing filenames, might be useful in earlier steps of this workflow already.

# Notes

## Data processing notes

Pybids gave `unknown locale UTF-8` issue, fixed by adding values to path variable upon conda env startup (and removing upon deactivation), see: https://coderwall.com/p/-k_93g/mac-os-x-valueerror-unknown-locale-utf-8-in-python

Need to build eprime code into a script, currently in a notebook

Remember for BIDS-validator:
- Delete T1w file and rename defaced file to original T1w name
- add events.tsv
- SliceTiming required for all functionals - need to find a better place to implement this in future. For now just a script to run through all files and update json with precalculated slice times based on known parameters. This is needed since Philips does not supply slice timing information in PAR/rec or DICOM.


## Data quality notes

#### Problem subjects: EPI acquisition

- **sub-008** - last scan not finished (recon issue), need to rescan
- **sub-014** - last scan not finished (recon issue), need to rescan

#### Problem subjects: scanphyslog

UPDATE 13 January 2020:
*I incorrectly attributed the emotion1 scan restart problem to sub-015, it was actually sub-016 as confirmed on RIS system (see photos). This also explains "extra small file" previously mentioned in sub-016 dataset*

- **sub-015** - emotion1 scan was started twice, because heartrate sensor was not correctly recording. Second time was fine. need to inspect earlier scans for same issue. also, looks like scanphyslog didn't restart and kept loggin to same file, resulting in >12mb log file. need to inspect
- **sub-016** - All image data fine. Have to look at this data on RIS since scanphyslog has one extra small file which doesn't seem to fit

#### Problem subjects: T1w display

- **sub-017** - when displaying in Mango, top slices of brain are cut off, need to check if this is a viewer issue or acquisition issue. Need to check other subjects as well.

#### Steps done for now (8 Jan 2020):

- **sub-008** - deleted from organised data and BIDS data
- **sub-014** - deleted from organised data and BIDS data
- **sub-015** - removed from BIDS data, kept in organised data separate folder, scanphyslog renamed, need to inspect
- **sub-016** - removed from BIDS data, kept in organised data separate folder, scanphyslog NOT renamed, need to inspect

#### Steps to do (UPDATE 13 Jan 2020):

- **sub-015** - move back to BIDS and ensure everything is correctly named and converted
- **sub-016** - inspect everything, ensure this is correctly named and converted (ignore small file).
